# MultiModal Modeling Image To Text

https://huggingface.co/docs/transformers/master/model_doc/vision-encoder-decoder#transformers.TFVisionEncoderDecoderModel
https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/VisionTransformer/Quick_demo_of_HuggingFace_version_of_Vision_Transformer_inference.ipynb


In [1]:
!sudo apt update
!sudo apt install -y python3-dev python3-pip python3-venv

Hit:1 https://packages.microsoft.com/repos/azure-cli focal InRelease
Hit:2 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:3 https://cli.github.com/packages focal InRelease                          
Hit:4 http://ppa.launchpad.net/xapienz/curl34/ubuntu focal InRelease           
Hit:5 http://security.ubuntu.com/ubuntu focal-security InRelease               
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease                     
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease                 
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease           
Hit:9 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease
Hit:10 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease   
Reading package lists... Done
Building dependency tree       
Reading state information... Done
36 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       

In [2]:
!pip install -U ipykernel transformers pillow requests urllib3 certifi idna pyyaml torch tensorflow ipywidgets jupyter jupyter_contrib_nbextensions --quiet

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
from transformers import AutoFeatureExtractor, AutoTokenizer, TFVisionEncoderDecoderModel
from PIL import Image
import requests
import torch
import tensorflow

2022-02-15 15:21:29.822377: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-15 15:21:29.822419: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:

feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
decoder_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [5]:

# initialize a bert2gpt2 from a pretrained BERT and GPT2 models. Note that the cross-attention layers will be randomly initialized
model = TFVisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    "google/vit-base-patch16-224-in21k", "gpt2"
)

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
img = Image.open(requests.get(url, stream=True).raw)

# forward
pixel_values = feature_extractor(images=img, return_tensors="tf").pixel_values  # Batch size 1
decoder_input_ids = decoder_tokenizer("Linda Davis", return_tensors="tf").input_ids  # Batch size 1
outputs = model(pixel_values=pixel_values, decoder_input_ids=decoder_input_ids)

2022-02-15 15:21:51.733378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-15 15:21:51.733428: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-15 15:21:51.733461: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces_f9f5a2): /proc/driver/nvidia/version does not exist
2022-02-15 15:21:51.734414: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel

In [6]:
outputs.logits.shape

TensorShape([1, 3, 50257])

In [7]:
import tensorflow as tf

predictions = tf.math.softmax(outputs.logits, axis=-1)
print(predictions)

tf.Tensor(
[[[1.4185678e-03 2.6717638e-03 2.4844165e-04 ... 5.7827606e-07
   2.0972002e-06 1.5366605e-03]
  [2.8044521e-04 1.1812281e-02 1.4363402e-05 ... 1.9583121e-08
   1.0469118e-05 3.4460629e-04]
  [1.3529514e-04 4.7747428e-03 1.2293266e-05 ... 3.2709604e-08
   2.5662534e-06 1.2595564e-03]]], shape=(1, 3, 50257), dtype=float32)


In [ ]:

outputs = model(pixel_values=pixel_values, decoder_input_ids=decoder_input_ids, labels=decoder_input_ids)
loss, logits = outputs.loss, outputs.logits
print(loss)
print(logits)

In [ ]:
model.save_pretrained("vit-gpt2")

## Load VIT-GPT2

In [ ]:
from transformers import AutoFeatureExtractor, AutoTokenizer, TFVisionEncoderDecoderModel
from PIL import Image
import requests
import torch
import tensorflow

In [18]:
feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
decoder_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [19]:
# save and load from pretrained
model = TFVisionEncoderDecoderModel.from_pretrained("vit-gpt2")

# generation
generated = model.generate(pixel_values, decoder_start_token_id=model.config.decoder.bos_token_id)

All model checkpoint layers were used when initializing TFVisionEncoderDecoderModel.

All the layers of TFVisionEncoderDecoderModel were initialized from the model checkpoint at vit-gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFVisionEncoderDecoderModel for predictions without further training.


In [20]:
image_of_car = "https://d1ix0byejyn2u7.cloudfront.net/drive/images/uploads/headers/ws_cropper/1_0x0_1520x1040_0x520_car-body-types-mazda-cx5.jpg"
img = Image.open(requests.get(image_of_car, stream=True).raw)
pixel_values = feature_extractor(images=img, return_tensors="tf").pixel_values  # Batch size 1
generated = model.generate(pixel_values, decoder_start_token_id=model.config.decoder.bos_token_id)

In [21]:
decoder_tokenizer.batch_decode(generated, skip_special_tokens=True)

['\nThe U.S. Department of Justice has filed a lawsuit against the company that owns the']

In [ ]:
image_of_number = "https://www.researchgate.net/profile/Thomas-Wiatowski/publication/287853768/figure/fig5/AS:667034937462785@1536044930131/Handwritten-digits-from-the-MNIST-data-set-5-If-f-denotes-the-image-of-the-handwritten.jpg"
img = Image.open(requests.get(image_of_number, stream=True).raw)
pixel_values = feature_extractor(images=img, return_tensors="tf").pixel_values  # Batch size 1
generated = model.generate(pixel_values, decoder_start_token_id=model.config.decoder.bos_token_id)
decoder_tokenizer.batch_decode(generated, skip_special_tokens=True)

## VisionEncoderDecoderModel

In [ ]:
pip install sentencepiece --quiet

In [2]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests
from PIL import Image
import torch

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:

# load image from the IAM dataset
url = "https://fki.tic.heia-fr.ch/static/img/a01-122-02.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

# training
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

pixel_values = processor(image, return_tensors="pt").pixel_values
text = "hello world"
labels = processor.tokenizer(text, return_tensors="pt").input_ids
outputs = model(pixel_values=pixel_values, labels=labels)
loss = outputs.loss


In [4]:

# inference (generation)
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
generated_text

2022-02-14 20:10:50.240141: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-14 20:10:50.240178: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


'industry, " Mr. Brown commented icily. " Let us have a'

## VisionEncoderDecoderConfig

In [6]:
from transformers import BertConfig, ViTConfig, VisionEncoderDecoderConfig, VisionEncoderDecoderModel

# Initializing a ViT & BERT style configuration
config_encoder = ViTConfig()
config_decoder = BertConfig()

config = VisionEncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)

# Initializing a ViTBert model from a ViT & bert-base-uncased style configurations
model = VisionEncoderDecoderModel(config=config)

# Accessing the model configuration
config_encoder = model.config.encoder
config_decoder = model.config.decoder
# set decoder config to causal lm
config_decoder.is_decoder = True
config_decoder.add_cross_attention = True

# Saving the model, including its configuration
model.save_pretrained("my-model")

# loading model and config from pretrained folder
encoder_decoder_config = VisionEncoderDecoderConfig.from_pretrained("my-model")
model = VisionEncoderDecoderModel.from_pretrained("my-model", config=encoder_decoder_config)

In [ ]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
img = Image.open(requests.get(url, stream=True).raw)
generated_ids = model.generate(pixel_values)
